### 크런치 프라이스 전체 상품 크롤링

In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from time import sleep
import math
import warnings
import threading
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
# 로그인
driver.get("http://gdadmin.crunchprice.com/base/login.php")
driver.find_element_by_id("login").send_keys("dooyeoung") 
driver.find_element_by_name("managerPw").send_keys("dooyeoung123!")
driver.find_element_by_css_selector("input[type=submit]").click() 

# 상품관리 메뉴 이동
while(True):
    goods = driver.find_elements_by_id("menu_goods")
    if len(goods) > 0: 
        break

trcnt = 200
driver.get("http://gdadmin.crunchprice.com/goods/goods_list.php?page={}&pageNum={}".format(1, trcnt))

total_cnt = driver.find_element_by_css_selector("#frmSearchGoods > div.table-header > div.pull-left > strong:nth-child(2)")
total_cnt = int(total_cnt.text.replace(",",""))
totla_pages = math.ceil(total_cnt / trcnt)

In [3]:
def crawl(start, end):
    print(start, end)
    driver = webdriver.PhantomJS()
    
    driver.get("http://gdadmin.crunchprice.com/base/login.php")
    driver.find_element_by_id("login").send_keys("dooyeoung") 
    driver.find_element_by_name("managerPw").send_keys("dooyeoung123!")
    driver.find_element_by_css_selector("input[type=submit]").click() 

    # 상품관리 메뉴 이동
    while(True):
        goods = driver.find_elements_by_id("menu_goods")
        if len(goods) > 0: 
            break
            
     
    #total = 0 
    for cur_page in range(start, end): 
        
        next_page = "http://gdadmin.crunchprice.com/goods/goods_list.php?page={}&pageNum={}".format(cur_page, trcnt)
        driver.get(next_page)
        
        trs = driver.find_elements_by_css_selector("#frmList > table > tbody > tr")
        
        if len(trs) == 0:
            break
            
        for tr in trs:
            html= tr.get_attribute('innerHTML')
            soup = BeautifulSoup(html) 
            tds = soup.find_all("td")
            seq = tds[1].text
            code = tds[2].text    
            img = tds[3].find('a')['href']
            name = tds[4].find('div').text
            price = tds[5].text.strip()
            seller = tds[6].text
            link = tds[11].find('a')['href']  

            result.append([seq, code, img, name, price, seller, link])

        #total += len(trs) 

        
        sleep(0.5) 
        #del trs 
        
    #driver.close() 

In [ ]:
thread_num = 10
np = math.ceil(totla_pages / thread_num)
for idx in range(1, 11):
     
    # 스레드당 담당할 페이지
    start = (idx-1) * np + 1
    end = np * idx + 1
    
    result = []
 
    th = threading.Thread(target=crawl, args=(start, end))
    th.start() 

In [18]:
df = pd.DataFrame(result)
df.to_csv("items.csv", index=False)